In [ ]:
from typing import Dict

import polars as pl

In [ ]:
RAW_DATA: str = "data_2021_main_dvs-soti_v1.csv"
TOOLS_COL_PREFIX: str = "ToolsForDV_"
ID_COL: str = "chronID"

TOOLS_MAP: Dict[str, str] = {
    "D3": "D3.js",
    "GoogleDataStudio": "Google Data Studio",
    "KeplerGL": "kepler.gl",
    "PowerBI": "Power BI",
    "WebComponents": "Web Components",
    "PenPaper": "Pen & paper",
    "PhysicalMaterials": "Physical materials (other than pen and paper)",
    "P5orProcessing": "P5/Processing",
}

In [ ]:
df = pl.scan_csv(RAW_DATA)

In [ ]:
# df.collect().shape
# type(df)
# df.fetch(5)
df.fetch(1)
# df.columns

In [ ]:
q_tools = df.select([pl.col(ID_COL), pl.col(f"^{TOOLS_COL_PREFIX}.*$")]).drop(
    f"{TOOLS_COL_PREFIX}Other__"
)

In [ ]:
# tools_cols = [
#     col
#     for col in df.columns
#     if col.startswith(TOOLS_COL_PREFIX) and col != f"{TOOLS_COL_PREFIX}Other__"
# ]
# tools_cols = [
#     pl.col(col)
#     for col in df.columns
#     if (col.startswith(TOOLS_COL_PREFIX) and col != f"{TOOLS_COL_PREFIX}Other__")
#     or col == "chronID"
# ]
# tools_cols

# q_tools = df.select(tools_cols)

In [ ]:
print(q_tools.describe_plan())

In [ ]:
print(q_tools.describe_optimized_plan())

In [ ]:
# df_tools = q_tools.collect(no_optimization=False)
# df_tools = q_tools.collect(no_optimization=True)

# More info:
# - https://github.com/pola-rs/polars/issues/1659#issuecomment-956499225

# df_tools = q_tools.collect(projection_pushdown=True)
df_tools = q_tools.collect(projection_pushdown=False)

In [ ]:
# type(df_tools)
df_tools.shape

In [ ]:
df_tools[ID_COL].n_unique()

In [ ]:
df_tools.head()

In [ ]:
df_tools.tail()

In [ ]:
df_tools[TOOLS_COL_PREFIX].value_counts()

In [ ]:
df_tools["ToolsForDV_ArcGIS"].value_counts()

In [ ]:
df_tools["ToolsForDV_D3"].value_counts()

In [ ]:
df_tools.null_count()

In [ ]:
predicate = pl.all().exclude([ID_COL, TOOLS_COL_PREFIX]).str.contains("^$")

# q_count_tools = df_tools.lazy().select(
#     [
#         predicate.sum().suffix("_notuse"),
#         predicate.is_not().sum().suffix("_use"),
#     ]
# )

# q_count_tools.collect().transpose(
#     include_header=True, header_name="tool", column_names=["count"]
# )

q_use_tools = df_tools.lazy().select(predicate.is_not().sum())
# q_use_tools.collect()

q_not_use_tools = df_tools.lazy().select(predicate.sum())
# q_not_use_tools.collect()

In [ ]:
df_use_tools = q_use_tools.collect().transpose(
    include_header=True, header_name="tool", column_names=["use_count"]
)

df_not_use_tools = q_not_use_tools.collect().transpose(
    include_header=True, header_name="tool", column_names=["not_use_count"]
)

# df_use_tools.head()
# df_not_use_tools.head()

In [ ]:
df_count_tools = df_use_tools.join(df_not_use_tools, on="tool")

df_counts_tools = df_count_tools.with_columns(
    [
        pl.col("tool").str.slice(start=len(TOOLS_COL_PREFIX)),
        (pl.col("use_count") + pl.col("not_use_count")).alias("total_count"),
    ]
)

In [ ]:
df_counts_tools["tool"].to_list()

In [ ]:
# Adapted from: https://pola-rs.github.io/polars-book/user-guide/howcani/apply/udfs.html#apply-custom-functions
def rename_tools(tool: str) -> str:
    return TOOLS_MAP.get(tool, tool)


df_counts_tools = df_counts_tools.with_column(pl.col("tool").apply(rename_tools))

In [ ]:
df_counts_tools.head()

In [ ]:
df_counts_tools.tail()

In [ ]:
df_counts_tools["total_count"].value_counts()

In [ ]:
# df_counts_tools.to_csv("tools_counts.csv")

---